In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [45]:
## LOding the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
#Preprocessing the dataset
###Drop irrelevant columns
data=df
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [47]:
# ## Encode categorical variables
label_encoder= LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])


c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [48]:
#One hot encoding for categorical variables for Geography 
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geography_encoded = ohe.fit_transform(data[['Geography']])

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [49]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [50]:
# Convert the sparse matrix to a dense format and create a DataFrame
geography_df = pd.DataFrame(geography_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))
geography_df
# Concatenate the new DataFrame with the original data
data = pd.concat([data, geography_df], axis=1)
# Drop the original Geography column
data = data.drop(['Geography'], axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [51]:
#Save the encoders ----***************
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('ohe.pkl', 'wb') as f:
    pickle.dump(ohe, f)

In [52]:
# Divide the dataset into features and target variable
X = data.drop(['Exited'], axis=1)   
y = data['Exited']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.appl

In [53]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [54]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8000, 12), (2000, 12), (8000,), (2000,))

In [55]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

ANN Implementation


In [56]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import datetime


In [57]:
from tensorflow.keras.models import Sequential #ANN
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization


| Expression            | Meaning                        |
| --------------------- | ------------------------------ |
| `X_train.shape`       | `(num_samples, num_features)`  |
| `X_train.shape[1]`    | `num_features`                 |
| `(X_train.shape[1],)` | Tuple of input shape for Keras |


In [58]:
### Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),# Input layer with number of features and first hidden layer
    Dense(32,activation='relu'), # Second hidden layer
    Dense(16, activation='relu'),# Third hidden layer
    Dense(1, activation='sigmoid') # Output layer for binary classification
])


In [59]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
#optimizers 
from tensorflow.keras.optimizers import Adam 

# loss function
from tensorflow.keras.losses import BinaryCrossentropy

Adam_optimizer = Adam(learning_rate=0.01)

loss_function = BinaryCrossentropy()



In [61]:
# Compile the model
#model.compile(optimizer='adam', loss='Binary_crossentrophy', metrics=['accuracy'])
model.compile(optimizer=Adam_optimizer, loss=loss_function, metrics=['accuracy'])

In [62]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard,CSVLogger
## Setting up TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create a TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

#CSV logger
csv_logger = CSVLogger('training_log.csv', append=True, separator=';')


In [63]:
#Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [64]:
### TRain the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100,
                    callbacks=[early_stopping, tensorboard_callback, csv_logger])

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.4025 - accuracy: 0.8320 - val_loss: 0.3632 - val_accuracy: 0.8550
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3540 - accuracy: 0.8545 - val_loss: 0.3481 - val_accuracy: 0.8595
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3485 - accuracy: 0.8574 - val_loss: 0.3537 - val_accuracy: 0.8480
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3436 - accuracy: 0.8614 - val_loss: 0.3519 - val_accuracy: 0.8545
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3440 - accuracy: 0.8612 - val_loss: 0.3415 - val_accuracy: 0.8600
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3378 - accuracy: 0.8618 - val_loss: 0.3473 - val_accuracy: 0.8640
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3367 - accuracy: 0.8644 - val_loss: 0.3477 - val_accuracy: 0.8545

In [65]:
model.save('churn_model.h5')

c:\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
## Load tenssorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [67]:
%tensorboard --logdir logs/fit/20250704-100217
print("Model training completed and saved as 'churn_model.h5'.")
print(f"tensorboard running on port 6006 {http://localhost:6006}")


Reusing TensorBoard on port 6007 (pid 27916), started 1:34:33 ago. (Use '!kill 27916' to kill it.)

Model training completed and saved as 'churn_model.h5'.


NameError: name 'http' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss Curve")
plt.show()